<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/Vakhranyov_AY_Deep_learning_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом проекте я постарался с помощью предобученного берта сделать предсказания для текста на сентимент-анализ. Я взял датасет Sentiment Analysis Dataset.csv он есть здесь (https://github.com/vineetdhanawat/twitter-sentiment-analysis/blob/master/datasets/Sentiment%20Analysis%20Dataset.csv). Пояснений к нему, к сожалению, нет, поэтому точно сказать не могу, но, судя по всему, представленные в нём разбиты следующим образом: целевой класс (представлены единицей) — твиты с положительно-окрашенным мнением, всё остальное — представлено нулями. Помимо того, что я попробовал замерять лоссы на эпохах, также я использовал ф-меру для оценки модели.

В последнее время применение глубокого обучения для решения проблемы сентимент-анализа стало популярной темой исследований. Существуют различные архитектуры глубокого обучения и технологии, которые применяют для подобного рода анализа: эмбеддинги, автоэнкодеры, CNN, RNN, LSTM, применение аттеншн-механизма в RNN, MemNN, RecNN. Многие из этих методов глубокого обучения показали отличные результаты для различных задач сентимент-анализа. Если верить научным работам по теме, которые я просмотрел, то с развитием исследований и приложений глубокого обучения в ближайшем будущем появятся более интересные исследования применения глубокого обучения для сентимент-анализа.

In [92]:
!pip install transformers
import pandas as pd
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
data = pd.read_csv("Sentiment Analysis Dataset.csv", header= None)
max_len = 512

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [93]:
data = data[:500]
data.columns = ["id", "sentiment", "text"]

In [94]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.1, random_state=42)

In [95]:
from torch.utils.data import Dataset
class dataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):   
        text = str(self.texts[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(text, add_special_tokens=True,max_length=self.max_len, return_token_type_ids=False,padding='max_length', return_attention_mask=True, return_tensors='pt', truncation = True)
        return {'text': text, 'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'targets': torch.tensor(target, dtype=torch.long)}

In [96]:
train.text.head

<bound method NDFrame.head of 72       I think I may be too friendly...lol... o wel...
182     @adbert: &quot;#Video [Woody Woodpecker вЂ“ T...
131      you'll always be my one and only...  [12*23*...
410         A dog riding the bicycle http://bit.ly/gvMzD
193     @dandelionas is making fettucini and garlic b...
                             ...                        
106      really wanted Safina to pull out a win &amp;...
270       lol all these #robotpickuplines are hilarious 
348                            what the fucccckkkkkkkkkk
435      I dont want my sister to leave me for the wh...
102                                    not a cool night.
Name: text, Length: 450, dtype: object>

In [97]:
import numpy as np
def Dataloader(data, tokenizer, max_len, batch_size):
    ds = dataset(data.text.to_numpy(), data.sentiment.to_numpy(), tokenizer=tokenizer, max_len=max_len)
    return Dataloader(ds,tokenizer=tokenizer,max_len=max_len,batch_size=batch_size)

batch_size = 8
loader = Dataloader(train, tokenizer, max_len, batch_size)
test_loader = Dataloader(test, tokenizer, max_len, batch_size)

AttributeError: ignored

In [ ]:
model = AutoModel.from_pretrained('bert-base-uncased')

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self, bert, hidden_size=768, output_size=2):
        super().__init__()
        self.bert = bert
        self.classifier = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        pred = self.bert(x, attention_mask=(x!=0))
        return self.classifier(pred.pooler_output)

In [ ]:
epochs = 6
for param in model.parameters():
    param.requires_grad = False
loss = torch.nn.CrossEntropyLoss()
classifier = Classifier(model)
optimizer = torch.optim.Adam(classifier.classifier.parameters())

In [ ]:
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np
train_losses = list()
train_loss_per_epoch = list()
test_loss_per_epoch = list()
train_f1_scores_per_epoch = list()
test_f1_scores_per_epoch = list()
best_test_metric_value = float('inf')
for epoch in range(epochs):
    train_epoch_losses = list()
    test_epoch_losses = list()
    train_predictions = list()
    train_targets = list()
    test_predictions = list()
    test_targets = list()
    print(f'Epoch: {epoch}')
    classifier.train()
    for x, y in tqdm(loader):
        x = x.long()
        pred = classifier(x)
        train_loss = loss(pred, y)
        train_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_losses.append(train_loss.item())
        train_epoch_losses.append(train_loss.item())
        train_predictions.extend(pred.argmax(dim=1).tolist())
        train_targets.extend(y.tolist())
    classifier.eval()
    for x, y in tqdm(test_loader):
        x = x.long()
        with torch.no_grad():
            pred = classifier(x)
            test_loss = loss(pred, y)
        test_epoch_losses.append(test_loss.item())
        test_predictions.extend(pred.argmax(dim=1).tolist())
        test_targets.extend(y.tolist())
    train_loss_per_epoch.append(np.mean(train_epoch_losses))
    test_loss_per_epoch.append(np.mean(test_epoch_losses))
    train_f1_scores_per_epoch.append(f1_score(train_targets, train_predictions))
    test_f1_scores_per_epoch.append(f1_score(test_targets, test_predictions))
    print(test_f1_scores_per_epoch)
    f1 = f1_score(y, pred.argmax(dim=1))
    print(f'Epoch {epoch}')
    print(f'Loss: train {train_loss_per_epoch[-1]:.2f} | test {test_loss_per_epoch[-1]:.2f}')
    print(f'F1 Score: train {train_f1_scores_per_epoch[-1]:.2f} | test {test_f1_scores_per_epoch[-1]:.2f}')
    #if best_test_metric_value > test_f1_scores_per_epoch[-1]:
    #  best_test_metric_value = test_f1_scores_per_epoch[-1]
    #else:
    #  break

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(14, 12))
plt.plot(train_loss_per_epoch)
plt.grid()

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(14, 12))
plt.plot(test_loss_per_epoch)
plt.grid()

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(14, 12))
plt.plot(train_f1_scores_per_epoch)
plt.grid()

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(14, 12))
plt.plot(test_f1_scores_per_epoch)
plt.grid()

In [ ]:
print('Лучшая метрика на test:')
print('F1 test: ', max(test_f1_scores_per_epoch))

Я выдающимися знаниями особо никогда не блистал, поэтому для меня даже достаточно простая задача (если сравнивать с другими вариантами проектов) давалась непросто. Наверное, главная проблема, с которой я столкнулся — это подсчёт ф-меры на большом количестве данных на тесте. На сотнях и десятках тысяч я упорно получал нули после каждой эпохи. В причнах этой проблемы, к сожалению, мне разобраться не удалось. Я пробовал увеличивать количество эпох обучения, но результат не менялся. Уменьшать learning rate я не стал, потому что по умолчанию он и так стоит небольшой, вряд ли это помогло бы с решением проблемы. В результате, относительного успеха удалось добиться, снизив объём данных до 2000.
Но здесь, само собой, были свои проблемы.
Например, ф-мера на трейне была довольно низкая, а лоссы на трейне вообще вели себя довольно хаотично. Также, как видно на графиках, на 6 (последней) эпохе резко портятся метрики, что, вероятнее всего, свидетельствует о переобучении модели. Думаю, что на таком количестве данных оптимально было бы остановиться на 5 эпохах.
Тем не менее, мне удалось получить ф-меру равную 0.77 на тесте, что является достаточно хорошим результатом.